<a href="https://colab.research.google.com/github/lty0858/AI4H2022/blob/main/2024_11_03_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#影音下載 (使用 yt-dlp)
▋ yt-dlp 專案
* https://github.com/yt-dlp/yt-dlp

▋ 搭配使用 FFmpeg
* https://www.ffmpeg.org/

▋ 指令參考
* https://cheat.sh/yt-dlp

▋ 詳細說明及問題反應
* [雄:Toots:yt-dlp 下載 Youtube 或是其它網站的影音](https://gsyan888.blogspot.com/2023/03/tools-yt-dlp.html)

In [ ]:
#@title yt-dlp 影音下載 { vertical-output: true }

#@markdown <font size="6">👈</font> <b>按左側的執行鈕即可以開始下載 ... But ... 請先設定底下的自訂參數</b>

#@markdown ▋ <b>影音檔的網址</b> (不只YouTube影片，也可是播放清單、Facebook 影片、Twitter ...)
url = "https://www.youtube.com/watch?v=g8mvtwXoe_w" #@param {type:"string"}

#@markdown ▋ <b>輸出為哪一種格式</b>（mp3、mp4）
outputFormat = 'mp3' #@param ['mp3', 'mp4']

#@markdown ▋ <b>影片畫質</b> (設定值為期待的畫質, 需視來源是否有提供)
videoResolution = '最高畫質' #@param ['最高畫質', '1080p', '720p', '480p', '320p', '240p']

#@markdown ▋ <b>速度倍率</b>（例如  1: 正常, 0.8: 變慢 1.2: 變快）
tempo = 1 #@param {type:"number"}

#@markdown ▋ <b>是否將所有 MP3 合併為一個音檔</b> (打勾會將清單的所有 MP3 接成一個檔案)
enable_mp3_concat = False #@param { type: 'boolean' }

#@markdown ▋ <b>是否立即下載</b>
start_downloading_immediately = True #@param { type: 'boolean' }

#@markdown ---
#@markdown <center><p><font color="blue">以下為指令輸出內容</p></center>


# Install + Import + Config
try: from yt_dlp import YoutubeDL
except :
  print('🏁 >> 安裝 yt_dlp 相關工具, 請稍候 ...')
  #! pip -q install yt_dlp  > /dev/null 2>&1
  #最新的測試版
  #!pip install -U --pre "yt-dlp[default]"
  !pip -q install yt-dlp@git+https://github.com/yt-dlp/yt-dlp.git > /dev/null 2>&1

try: from slugify import slugify
except:
  print('install python-slugify ...')
  ! pip -q install python-slugify

from yt_dlp import YoutubeDL
import re
import os
import urllib.request
from slugify import slugify

import google

#url = "https://voca.ro/15HVH0YvIaa6"
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ&list=PLelNvYGEtsV8TpwxL4t7GTTG-7qALZqol'
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ'
#url = 'vocaroo-台語.mp3'

tempFolder = '.' #暫存的資料夾(工作目錄、下載的影音、剛轉好的文字檔)
output_path = outputFormat
title = ''
outputFileList = [] #記錄下載好的檔名清單

resolution = ''.join(re.findall(r'\d+', videoResolution))

if not os.path.exists(output_path):
  os.mkdir(output_path)
else :
  for filename in os.listdir(output_path):
    filepath = os.path.join(output_path, filename)
    try :
      os.remove(filepath)
    except:
      pass


#檢查輸出文字檔的資料夾是否存在, 不存在就改用預設值
if re.sub('\s', '', output_path) == '' :
  #如果沒有輸入，就輸出到預設的暫存資料夾
  output_path = tempFolder
else :
  if os.path.exists(output_path) :
    if not os.path.isdir(output_path) :
      print(output_path + ' 不是資料夾哦! 改用預設值: '+tempFolder)
      output_path = tempFolder
  else :
    print('指定的輸出目錄 '+output_path+' 不存在, 改用預設值: '+tempFolder)
    output_path = tempFolder

def getYoutubePlaylistInfo_ydl(url, extract_flat=False) :
  """
  使用 yt_dlp 由 Youtbue 播放清單網址擷取各影片的資訊
  :param url: Youtube 影片播放清單網址
  :param extract_flat: 是否不要再將清單中影片一個個擷取資訊(預設要,影片多時會很慢)
  """
  ydl_opts = {
      'no_warnings':True,
      'ignoreerrors': True,
      'quiet':True,
      'extract_flat': extract_flat
  }
  with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    #return (info_dict.get('entries', None)) #entries 為清單中的影片, 'webpage_url' 為影片網址
    return info_dict

def getYoutubeVideoInfo_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 網址擷取影片的標題字
  :param url: Youtube 影片網址
  """
  ydl_opts = {
      'no_warnings':True,
      'ignoreerrors': True,
      #'dump_single_json': True,
      'quiet':True
  }
  with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    #return(info_dict.get('title', None))
    return info_dict

def downloadFromYoutube_ydl(url, outputFormat, outputFilename, isYT=True):
  """
  使用 yt_dlp 由 Youtbue 網址下載影片並存檔
  :param url: Youtube 影片網址
  :param outputFormat: 儲存的格式
  :param outputFilename: 檔名
  :param isYT: 是否為 Youtube video
  """
  #filename為去掉副檔名的路徑，以免變成 xxx.mp3.mp3
  filename = os.path.splitext(outputFilename)[0]
  if outputFormat=='mp3' :
    ydl_opts = {
        'no_warnings':True,
        'quiet':True,
        #'outtmpl':f'{filename}.%(ext)s',
        'outtmpl':filename,
        'keepvideo':False,
        'ignoreerrors':True,
        'format': 'bestaudio/best',
        'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
        #  'preferredquality': '192',
        }],
        'overwrites':True
    }
  else :
    ydl_opts = {
        'no_warnings':True,
        'quiet':True,
        'outtmpl':f'{filename}.%(ext)s',
        #'outtmpl':filename,
        'keepvideo':False,
        'ignoreerrors':True,
        #'format': 'bestvideo+bestaudio/best',
        #'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio',
        #'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]',
        'format': "bestvideo[vcodec~='^((he|a)vc|h26[45])']+bestaudio[ext=m4a]",
        'merge_output_format': 'mp4',
        'overwrites':True
    }
    if resolution != '':
      ydl_opts['format'] = f'bestvideo[height<={resolution}]+bestaudio/best[height<={resolution}]'
    if not isYT:
      del ydl_opts['merge_output_format']

  tryAgain = False
  try:
    with YoutubeDL(ydl_opts) as ydl:
      code = ydl.download(url)
      if code==1 :
        tryAgain = True
  except:
    tryAgain = True

  if tryAgain :
    print('\n > 重試中，請稍候...')
    del ydl_opts['format']
    with YoutubeDL(ydl_opts) as ydl:
      ydl.download(url)

def getVocarooMP3URL(url) :
  """
  Get the MP3 URL from Vocaroo record share URL
  :param url: Vocaroo url
  """
  vocarooMP3Base = 'https://media.vocaroo.com/mp3/'
  regex = re.compile(r'(https\:\/\/voca\.ro|https\:\/\/vocaroo\.com)\/(\w{12})')
  match = regex.search(url)
  if match :
    url = vocarooMP3Base+match.group(2)
  return url

#
def getFilenameFromTitle(title) :
  """
  convert title to valid filename
  :param title: text to slugify
  """
  if len(title) > 100:
    title = title[:100]
  return slugify(title, allow_unicode=True, lowercase=False)

def getOutputTextFilename(filename) :
  fPath = os.path.relpath(output_path)
  return os.path.join(fPath, filename+'.'+outputFormat)

def isOutputTextFileExists(filename) :
  return os.path.exists(getOutputTextFilename(filename))

def removeAudioFile(filename) :
  if os.path.exists(filename) :
    os.remove(filename)

def downloadFile(urlOrFile) :
  # 分網址或是Colab儲存空間檔案處理
  if re.search('https\:\/\/', urlOrFile) :
    isPlayList = False
    # Youtube 分播放清單跟單一影片處理
    if re.search('youtube\.|youtu\.', urlOrFile) :
      # 建立 Playlist 物件
      #pList = Playlist(urlOrFile)
      print('\n🏁 >> 分析連結內容 ...')
      if re.search('playlist', urlOrFile) :
        videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile, extract_flat=True)
        isPlayList = True
        #try : title = pList.title
        try : videos = videoInfo['entries']
        except :
          isPlayList = False
      else :
        isPlayList = False
        videoInfo = getYoutubeVideoInfo_ydl(urlOrFile)
      #處理 Youtube 播放清單
      if isPlayList :
        title = videoInfo['title']
        print('\n🏁 >>處理 Youtube 播放清單影片: '+title)
        for video in videos :
          if video is None :
            print('\n>>>>> [跳過] ... 此影片無法抓取，可能不是公開的影片 ...')
          else :
            title = video['title'] #取得Youtube影片的標題字
            if video['uploader'] is None and video['uploader_id']  is None and video['uploader_url'] is None :
              print('\n>>>>> [跳過] ... 此影片不是公開的影片( ' + video['url'] + ' ) ...')
            else :
              print('\n>>>>> [下載] '+ title)
              #用影片的標題當主檔名，但需將不適合的字置換或去除
              #convert title to valid filename
              outputFilename = getFilenameFromTitle(title)
              outputFilename = f'{outputFilename}.{outputFormat}'
              outputFilename = os.path.join(output_path, outputFilename)
              #downloadFromYoutube_ydl(video['webpage_url'], outputFormat, outputFilename)
              downloadFromYoutube_ydl(video['url'], outputFormat, outputFilename)
              #將輸出的檔名列入結果清單中，方便後續壓縮、下載
              if os.path.exists(outputFilename) :
                outputFileList.append(outputFilename)
        #title = pList.title #供製作下載壓縮檔用的主檔名
        title = videoInfo['title']
      else :
        # Youtube single video
        if videoInfo is not None and videoInfo['title'] is not None:
          title = videoInfo['title']
          print('\n>>>>> [下載] '+ title)
          outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
          outputFilename = f'{outputFilename}.{outputFormat}'
          outputFilename = os.path.join(output_path, outputFilename)
          downloadFromYoutube_ydl(urlOrFile, outputFormat, outputFilename)
          #將輸出的檔名列入結果清單中，方便後續壓縮、下載
          if os.path.exists(outputFilename) :
            outputFileList.append(outputFilename)
        else:
          title = None
    else :
      #非Youtube的網址
      isPlayList = False
      if re.search('voca\.ro|vocaroo\.com', urlOrFile) :
        # Vocaroo or other web audio
        urlOrFile = getVocarooMP3URL(urlOrFile) #取得儲存 Vocaroo 的語音檔網址
        title = "vocaroo-"+os.path.basename(urlOrFile) #用 Vocaroo 的 id 當標題字
        outputFilename = title #用 vocaroo-xxxxxxxxxxx 當輸出的主檔名
        outputFilename = os.path.join(output_path, f'{outputFilename}.mp3')
        urllib.request.urlretrieve(urlOrFile, outputFilename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        if os.path.exists(outputFilename) :
          outputFileList.append(outputFilename)
      else :
        # other website
        #videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
        videoInfo = getYoutubeVideoInfo_ydl(urlOrFile)
        title = videoInfo['title']
        print('\n>>>>> [下載] '+ title)
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        outputFilename = f'{outputFilename}.{outputFormat}'
        outputFilename = os.path.join(output_path, outputFilename)
        downloadFromYoutube_ydl(urlOrFile, outputFormat, outputFilename, False)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        if os.path.exists(outputFilename) :
          outputFileList.append(outputFilename)
        #! yt-dlp -q --force-overwrites -x --audio-format mp3 -o {audioFile} {url}
  return title

#解析 .txt 檔案中的清單，一行行進行去抓影音檔案
def parseTxtFileAndDownload(txtFile) :
  if os.path.exists(txtFile) :
    file = open(txtFile, 'r')
    lines = file.readlines()
    file.close()
    #print(len(lines))
    for line in lines :
      line = re.sub('\r|\n', '', line, re.MULTILINE) #去掉所有換行字元
      downloadFile(line)
  else :
    print('找不到檔案: '+txtFile)

if (re.search('https\:\/\/', url) is None) and (not re.search('\.txt$', url, re.IGNORECASE) is None) :
  #解析 .txt 檔中的清單後，再進行下載
  parseTxtFileAndDownload(url)
  title = os.path.splitext(os.path.basename(url))[0] #去掉資料夾名稱及附檔名，只取主檔名
else :
  #直接下載 url 指定的網址或是檔案
  try:
    title = downloadFile(url)
  except Exception as err:
    print('\n🏁 >>>> 下載時發生錯誤 ...')
    #print(f"Unexpected {err=}, {type(err)=}")

if len(outputFileList)>0 :
  #print(outputFileList)

  #檢查是否要調整 mp3 的速度並執行
  if tempo!=1 :
    print(f'\n🏁 >> 調整速度為 {tempo} 倍速...')
    if outputFormat=='mp3' :
      tempFile = 'temp.mp3'
    else :
      tempFile = 'temp.mp4'
    for srcFile in outputFileList :
      if os.path.exists(srcFile) :
        print(f'>>> {srcFile}')
        if os.path.exists(tempFile) :
          !rm {tempFile}
        if outputFormat=='mp3' :
          cmd = f"ffmpeg -hide_banner -loglevel error -y -i \"{srcFile}\" -filter:a \"atempo={tempo}\" {tempFile}"
        else :
          cmd = f"ffmpeg -hide_banner -loglevel error -y -i \"{srcFile}\" -filter_complex \"[0:v]setpts=PTS/{tempo}[v];[0:a]atempo={tempo}[a]\" -map \"[v]\" -map \"[a]\" {tempFile}"

        os.system(cmd)
        if os.path.exists(tempFile) :
          !cp -f {tempFile} {srcFile}
          !rm {tempFile}
  #檢查是否要將多個 mp3 合併為一個音檔
  if outputFormat=='mp3' and enable_mp3_concat and len(outputFileList)>1 :
    #製作 ffmpeg 輸入的語法: -i '1.mp3' -i '2.mep' -i '3.mp3' ...
    #inList = " -i '"+"' -i '".join(outputFileList)+"'"
    inList = "|".join(outputFileList)
    #輸出檔名
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.mp3'
    print(f'\n🏁 >>> 合併為一個音檔')
    if os.path.exists(outputFilename) :
      !rm {outputFilename}
    #cmd = f"ffmpeg -hide_banner -loglevel error -y -f concat {inList} {outputFilename}"
    cmd = f"ffmpeg -hide_banner -loglevel error -y -i \"concat:{inList}\" {outputFilename}"
    os.system(cmd)
  #自動下載
  if start_downloading_immediately :
    print('\n🏁 download...')

    if len(outputFileList)>1 :
      if outputFormat=='mp3' and enable_mp3_concat :
        #已啟用將多個 mp3 合併為一個音檔
        print('\n🏁 下載檔案')
      else :
        #下載多個影音的，先壓縮到 .zip 中
        print('\n🏁 壓縮並下載檔案')
        outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
        ! zip -r {outputFilename} {output_path}
    else :
      print('\n🏁 下載檔案')
      outputFilename = outputFileList[0]

    google.colab.files.download(outputFilename)
  else :
    print('\n🏁 >> 已經完成任務，請打開側欄的 [檔案] 自行下載')
else :
  print('\n🏁 >>>>> 無法下載，請確認是可以公開存取的影音後再試 ...')


🏁 >> 安裝 yt_dlp 相關工具, 請稍候 ...

🏁 >> 分析連結內容 ...

>>>>> [下載] YouTube 語言設定對 NotebookLM 的影響

🏁 download...

🏁 下載檔案


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 原始筆記本
* https://tinyurl.com/gsyan-yt-dlp